In [ ]:
import pyodbc
import pandas as pd
import numpy as np
from tqdm import tqdm
from fuzzywuzzy import fuzz

In [ ]:
server = "HQIRS"
database = "threetc_irs"

cnxn = pyodbc.connect("DRIVER={SQL Server};SERVER="+server+";DATABASE="+database)
cursor = cnxn.cursor()

query = '''
select *
from inc_incident
where inc_property_type in (
    select CODE
    from Reporting_MENU_IRS_PROPERTY_TYPE
    where CATEGORY = 'Dwelling'
)
and inc_initial_incident_type = 'Fire - Building'
'''

incident_df = pd.read_sql(query, cnxn)

In [ ]:
file_path = "C:\\Users\\agozacan\\OneDrive - Humberside Fire and Rescue Service\\Fire Fatality Profiling\\Data\\Residential EPC Data\\"

In [ ]:
epc_df = pd.read_csv(file_path+"combined_epc_data.csv")

In [ ]:
epc_df.replace(np.nan, "", inplace=True)
incident_df.replace(np.nan, "", inplace=True)

In [ ]:
epc_addresses = []

for i in tqdm(range(len(epc_df))):

    string = " ".join(thing for thing in epc_df.iloc[i, :4])

    epc_addresses.append(string)

In [ ]:
inc_addresses = []

for i in tqdm(range(len(incident_df))):

    string = " ".join(thing for thing in incident_df.loc[i, ["inc_location_address", "inc_location_postcode"]]).replace(";", " ")

    inc_addresses.append(string)

In [ ]:
### MATCHES EPC ADDRESSES TO INCIDENT ADDRESSES. TAKES ABOUT 6 HOURS TO RUN

matching_indices = []

for i in tqdm(range(len(epc_addresses))):

    fuzz_ratios = []

    for j in range(len(inc_addresses)):

        fuzz_ratios.append(fuzz.token_set_ratio(epc_addresses[i], inc_addresses[j]))

    if max(fuzz_ratios) > 97:

        index = fuzz_ratios.index(max(fuzz_ratios))

    else:

        index = 0

    matching_indices.append(index)

In [ ]:
### ATTACH THE MATCHING INCIDENT INDICES (AT TIME OF DATA LOADING) TO THE EPC DATA

for epc_ind, inc_ind in enumerate(tqdm(matching_indices)):

    epc_df.loc[epc_ind, "ref"] = inc_ind

In [ ]:
# epc_df.to_csv(file_path+"epc_with_incs_v2.csv", index=False)

In [ ]:
epc_df[epc_df["ref"] != 0].merge(right=incident_df["inc_location_address"], left_on="ref", right_index=True, how="inner").head(50).sort_values(by="ref")